# Redes convolucionales

## Pooling

En el ejemplo de convnet, habrás notado que el tamaño de los mapas de características se reduce a la mitad después de cada capa `MaxPooling2D`. Por ejemplo, antes de las primeras capas `MaxPooling2D`, el mapa de características es de `26 × 26`, pero la operación de agrupación máxima lo reduce a la mitad a `13 × 13`. Esa es la función de `maxpooling`: reducir agresivamente la resolución de los mapas de características, de forma muy similar a las convoluciones a `strides`.

El `maxpooling` consiste en extraer ventanas de los mapas de características de entrada y generar el valor máximo de cada canal. Es conceptualmente similar a la convolución, excepto que en lugar de transformar parches locales mediante una transformación lineal aprendida (el núcleo de convolución), se transforman mediante una operación de tensor máximo codificada. Una gran diferencia con la convolución es que la agrupación máxima generalmente se realiza con ventanas de `2 × 2` y `stride` `2`, para reducir la muestra de los mapas de características en un factor de `2`. Por otro lado, la convolución generalmente se realiza con ventanas de `3 × 3` y sin `stride` (zancada 1).

- Se usan para hacer **downsampling del input feature** (reducir dimensionalidad espacial)
- Similar a convoluciones, pero no aplican un kernel sino que aplican **funcion 'max' (o average)**
- `Se fijan en el vecindario y se quedan con el promedio o con el máximo.`
- `Por defecto, tamaño 2x2 y stride 2` Si tengo un mapa de 32 x 32 me lo pasa a uno de 16 x 16
- `Las operaciones Pooling siempre se ponene detras de convolucional`
- Esto nos permite reducir el coste computacional
- Se puede hacer con Stride, pero es más común realizarlo por Pooling
- Max Pooling tienen más popularidad que el AVG pooling, ya que si dejo pasar los valores máximos, los termino perdiendo, se difimunan.


#### ¿Para qué?

-   Reducir el número de parámetros
-   Permite a ventanas posteriores aceptar input de areas más grandes en capas anteriores (mejora percepción)

#### En resumen

La razón para utilizar `maxpooling` es reducir el número de coeficientes del mapa de características a procesar, así como inducir jerarquías de filtros espaciales al hacer que las capas de convolución sucesivas miren ventanas cada vez más grandes (en términos de la fracción de la entrada original). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
import warnings
warnings.filterwarnings("ignore")

In [ ]:
"""
Based Model: Encargado de extraer las características de las imágenes.
"""
#BASE MODEL (Destinada a FE)

"""
32,(3,3) 
- 32 es el número de filtros que va a contener esa capa
- (3,3) es el tamaño del filtro
- input_shape=(32,32,3)
"""

convnet_nopooling = Sequential()
convnet_nopooling.add(layers.Conv2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
convnet_nopooling.add(layers.Conv2D(32,(3,3),activation='relu'))
convnet_nopooling.add(layers.Conv2D(64,(3,3),activation='relu'))
convnet_nopooling.add(layers.Conv2D(64,(3,3),activation='relu'))

"""
Top Model: Aplano características
A este Perceptron multicapa ya le entran las caracteríticas aplanadas del Base Model.
"""

#TOP MODEL

"""
Estoy aplanando las caracteristicas.
"""
convnet_nopooling.add(layers.Flatten())
convnet_nopooling.add(layers.Dense(512,activation='relu'))
convnet_nopooling.add(layers.Dense(10,activation='softmax'))

convnet_nopooling.summary()

In [ ]:
convnet_pooling = Sequential()

########################## BASE MODEL ###### ENCARGADO DE EXTRAER CARACTERÍSTICAS AUTOMÁTICAMENTE

"""
Debo de identificar el problema para poder implementar el bloque que corrresponda para realidad pooling
"""


#BLOQUE CONVOLUCIONAL 1
convnet_pooling.add(layers.Conv2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
convnet_pooling.add(layers.Conv2D(32,(3,3),activation='relu'))
convnet_pooling.add(layers.MaxPooling2D((2,2)))

#BLOQUE CONVOLUCIONAL 2
convnet_pooling.add(layers.Conv2D(64,(3,3),activation='relu'))
convnet_pooling.add(layers.Conv2D(64,(3,3),activation='relu'))
convnet_pooling.add(layers.MaxPooling2D((2,2)))

########################## TOP MODEL ###### ENCARGADO LLEVAR A CABO LA ######CLASIFICACIÓN
convnet_pooling.add(layers.Flatten())
convnet_pooling.add(layers.Dense(512,activation='relu'))
convnet_pooling.add(layers.Dense(10,activation='softmax'))
convnet_pooling.summary()

Con este modelo, cambiamos la cantidad de parámetros que maneja.
Podemos observar que luego de la capa convolucional `conv2d_6`, al entrar el `max_pooling`, me baja el shape de `28,28,32` a `14,14,32`

Luego de la `conv2d_8`, al aplicar la `max_pooling2d_2`, vuelvo a reducir de `10,10,64` a `5,5,64` Como resultado me da 1600 features en mi capa aplanda comparado contra los cientos de miles del modelo sin `maxpooling`

## Parches (Patch)

**Tamaño de los parches** (habitualmente 3x3 o 5x5)
-   Depth (profundidad, **número de filtros**) del mapa de features del output (32 y 64 en el ejemplo)

`Conv2D(output_depth,(patch_height,patch_width))`

## Comprendiendo los efectos de borde y el relleno (Padding)

Considere un mapa de características de `5 × 5` (25 mosaicos en total). Sólo hay `9` mosaicos alrededor de los cuales puedes centrar una ventana de `3 × 3`, formando una cuadrícula de 3 × 3. Por lo tanto, el mapa de características de salida será de `3 × 3`. Se reduce un poco: en este caso, exactamente dos mosaicos a lo largo de cada dimensión. Puedes ver este efecto de borde en acción en el ejemplo anterior: comienzas con entradas de `28 × 28`, que se convierten en `26 × 26` después de la primera capa de convolución.

![](./img/patch_1.png)

Si desea obtener un mapa de características de salida con las mismas dimensiones espaciales que la entrada, puede usar relleno. El relleno consiste en agregar una cantidad adecuada de filas y columnas a cada lado del mapa de características de entrada para que sea posible ajustar ventanas de convolución centrales alrededor de cada mosaico de entrada. Para una ventana de `3 × 3`, agrega una columna a la derecha, una columna a la izquierda, una fila en la parte superior y una fila en la parte inferior. Para una ventana de `5 × 5`, agrega dos filas

![](./img/patch_2.png)

En las capas de Conv2D, el relleno se puede configurar mediante el argumento padding, que toma dos valores: `valid`, que significa que no hay relleno (solo se usarán ubicaciones de ventana válidas) y `same`, que significa "relleno de tal manera que tener una salida con el mismo ancho y alto que la entrada”. El argumento de relleno por defecto es `valid`.

## Comprendiendo los avances de convolución (Strides)

El otro factor que puede influir en el tamaño de la producción es la noción de avances. Nuestra descripción de la convolución hasta ahora ha asumido que los mosaicos centrales de las ventanas de convolución son todos contiguos. Pero la distancia entre dos ventanas sucesivas es un parámetro de la convolución, llamado `stride`, que por defecto es `1`. 

Es posible tener convoluciones con `strides`: convoluciones con una `strides` superior a `1`. 
En la figura, puedes ver los parches extraídos por un Convolución `3 × 3` con `strides` `2` sobre una entrada de `5 × 5` (sin relleno).

Usar paso 2 significa que el `ancho` y el `alto` del mapa de características se reducen en un factor de `2` (además de cualquier cambio inducido por los efectos de borde). Las convoluciones `strides` rara vez se utilizan en modelos de clasificación, pero resultan útiles para algunos tipos de modelos.

![](./img/strides.png)

En los modelos de clasificación, en lugar de avances, tendemos a utilizar la operación de agrupación máxima (`maxpooling`) para reducir la muestra de mapas de características.

## TRABAJANDO CON REDES PRE-ENTRENADAS: TRANSFER LEARNING & FINE-TUNING

Un enfoque común y muy eficaz para el aprendizaje profundo en conjuntos de datos de imágenes pequeños es utilizar un modelo previamente entrenado (`pretrained models`). Un modelo previamente entrenado es un modelo que se entrenó previamente en un gran conjunto de datos, generalmente en una tarea de clasificación de imágenes a gran escala. 

Si este conjunto de datos original es lo suficientemente grande y general, la jerarquía espacial de las características aprendidas por el modelo previamente entrenado puede actuar efectivamente como un modelo genérico del mundo visual y, por lo tanto, sus características pueden resultar útiles para muchos problemas diferentes de visión por computadora, aunque Estos nuevos problemas pueden involucrar clases completamente diferentes a las de la tarea original. 

Por ejemplo, podría entrenar un modelo en `ImageNet` (donde las clases son principalmente animales y objetos cotidianos) y luego reutilizar este modelo entrenado para algo tan remoto como identificar muebles en imágenes. Esta portabilidad de las características aprendidas entre diferentes problemas es una ventaja clave del aprendizaje profundo en comparación con muchos enfoques de aprendizaje superficial más antiguos, y hace que el aprendizaje profundo sea muy eficaz para problemas de datos pequeños.

Usaremos la arquitectura `VGG16`, desarrollada por `Karen Simonyan` y `Andrew Zisserman` en 2014. Aunque es un modelo antiguo, alejado del estado actual del arte y algo más pesado que muchos otros modelos recientes, su arquitectura es similar a la que ya está familiarizado y es fácil de entender sin introducir ningún concepto nuevo. 

Este puede ser su primer encuentro con uno de estos nombres de modelos: `VGG`, `ResNet`, `Inception`, `Xception`, etc.

### Extracción de características con un modelo previamente entrenado

La extracción de características consiste en utilizar las representaciones aprendidas por un modelo previamente entrenado para extraer características interesantes de nuevas muestras. Luego, estas características se ejecutan a través de un nuevo clasificador, que se entrena desde cero.

Como vio anteriormente, las convnets utilizadas para la clasificación de imágenes constan de dos partes: comienzan con una serie de capas de agrupación y convolución, y terminan con un clasificador densamente conectado. La primera parte se llama base convolucional del modelo. En el caso de las convnets, la extracción de características consiste en tomar la base convolucional de una red previamente entrenada, ejecutar los nuevos datos a través de ella y entrenar un nuevo clasificador encima de la salida.

![](./img/pretrained_1.png)



¿Por qué reutilizar únicamente la base convolucional? ¿Podríamos reutilizar también el clasificador densamente conectado? En general, se debe evitar hacerlo. La razón es que las representaciones aprendidas por la base convolucional probablemente sean más genéricas y, por lo tanto, más reutilizables: los mapas de características de una convnet son mapas de presencia de conceptos genéricos sobre una imagen, que probablemente sean útiles independientemente de la computadora. 

Pero las representaciones aprendidas por el clasificador serán necesariamente específicas del conjunto de clases en las que se entrenó el modelo; solo contendrán información sobre la probabilidad de presencia de tal o cual clase en la imagen completa. Además, las representaciones que se encuentran en capas densamente conectadas ya no contienen información sobre dónde se encuentran los objetos en la imagen de entrada; Estas capas eliminan la noción de espacio, mientras que la ubicación del objeto todavía se describe mediante mapas de características convolucionales. Para problemas en los que la ubicación de los objetos importa, las funciones densamente conectadas son en gran medida inútiles.

Tenga en cuenta que el nivel de generalidad (y por lo tanto de reutilización) de las representaciones extraídas por capas convolucionales específicas depende de la profundidad de la capa en el modelo. Las capas que aparecen antes en el modelo extraen mapas de características locales y altamente genéricos (como bordes visuales, colores y texturas), mientras que las capas que están más arriba extraen conceptos más abstractos (como “oreja de gato” u “ojo de perro”). 

Entonces, si tu nuevo conjunto de datos difiere mucho del conjunto de datos en el que se entrenó el modelo original, es mejor que utilices solo las primeras capas del modelo para realizar la extracción de características, en lugar de utilizar toda la base convolucional.


El modelo VGG16, entre otros, viene preempaquetado con `Keras`. Puedes importarlo desde el módulo `keras.applications`. Muchos otros modelos de clasificación de imágenes (todos previamente entrenados en el conjunto de datos ImageNet) están disponibles como parte de `keras.applications`:

- Xcepción
- Resnet
- mobile Resnet
- Efficient net

etc.

![](./img/pretrained_networks.png)


## Diferencias

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*p-2QjvJ4nDCfn3F5oIxvYA.png)


#### **- Cargando el conjunto de datos y acondicionándolo como en la VGG**

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import imagenet_utils
from sklearn.preprocessing import LabelBinarizer

# Importando y normalizando el set de datos CIFAR10
print("[INFO]: Loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
labelNames = ["Avión", "Automóvil", "Pájaro", "Gato", "Ciervo", "Perro", "Rana", "Caballo", "Barco", "Camión"]

#One-hot encoding
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# IMPORTANTE: Se normalizan los datos como se normalizaron en el entrenamiento con ImageNet!!
trainX = imagenet_utils.preprocess_input(trainX)
testX = imagenet_utils.preprocess_input(testX)

#print(trainX.shape)
#print(trainY.shape)

#### **- Cargando la topología de CNN (base model)**

In [ ]:
#keras incluye varias arquitecturas
# VGG16, VGG19, ResNet50, Xception, InceptionV3, InceptionResNetV2, MobileNetV2, DenseNet, RasNet
# documentacion https://keras.io/applications/
# Visual Geometry Group 16 / 19 (numero de layers)
# 1 y 2 en la competicion ImageNet 2014
# Kernels pequeños de 3x3

from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet',
                 include_top=False, # No incluir el top model, i.e. la parte densa destinada a la clasificación (fully connected layers)
                 input_shape=(32,32,3))
base_model.summary()

#### **- Creando el top model y congelando TODAS las capas convolucionales (TRANSFER LEARNING)**

In [ ]:
# conectarlo a nueva parte densa
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

base_model.trainable = False # Evitar que los pesos se modifiquen en la parte convolucional -> TRANSFER LEARNING
pre_trained_model = Sequential()
pre_trained_model.add(base_model)
pre_trained_model.add(layers.Flatten())
pre_trained_model.add(layers.Dense(256, activation='relu'))
pre_trained_model.add(layers.Dense(10, activation='softmax'))

pre_trained_model.summary()

#### **- Entrenando la solución**

In [ ]:
# Import the necessary packages
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
#from google.colab import drive

BASE_FOLDER = './data/'

# Compilar el modelo
print("[INFO]: Compilando el modelo...")
pre_trained_model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08), metrics=["accuracy"]) 

# Entrenamiento de la red
print("[INFO]: Entrenando la red...")
H_pre = pre_trained_model.fit(trainX, trainY, batch_size=128, epochs=20, validation_split=0.2)

# Almaceno el modelo en Drive
# Montamos la unidad de Drive
#drive.mount('/content/drive') 
# Almacenamos el modelo empleando la función mdoel.save de Keras
pre_trained_model.save(BASE_FOLDER+"deepCNN_CIFAR10_pretrained.h5") #(X)

# Evaluación del modelo
print("[INFO]: Evaluando el modelo...")
# Efectuamos la predicción (empleamos el mismo valor de batch_size que en training)
predictions = pre_trained_model.predict(testX, batch_size=128)
# Sacamos el report para test
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)) 

# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 20), H_pre.history["loss"], label="train_loss")
plt.plot(np.arange(0, 20), H_pre.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 20), H_pre.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 20), H_pre.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

## Creando el top model y descongelando bloques convolucionales (FINE TUNING)

Otra técnica ampliamente utilizada para la reutilización de modelos, complementaria a la extracción de características, es el `fine tuning`. 

Cuando el dominio destino es poco parecido al dataset del dominio original se aplica la técnica de `Fine Tuning` a diferencia del `Transfer Learning`

El `fine tuning` consiste en descongelar algunas de las capas superiores de una base de modelo congelada utilizada para la extracción de características y entrenar conjuntamente tanto la parte recién agregada del modelo (en este caso, el clasificador completamente conectado) como estas capas superiores. Esto se denomina ajuste fino porque ajusta ligeramente las representaciones más abstractas del modelo que se reutiliza para hacerlas más relevantes para el problema en cuestión.

Como se ha mencionado anteriormente es necesario congelar la base de convolución de `VGG16` para poder entrenar un clasificador inicializado aleatoriamente en la parte superior. Por la misma razón, solo es posible ajustar las capas superiores de la base convolucional una vez que el clasificador superior ya ha sido entrenado. 

Si el clasificador aún no está entrenado, la señal de error que se propaga a través de la red durante el entrenamiento será demasiado grande y las representaciones aprendidas previamente por las capas que se están ajustando se destruirán. Por tanto, los pasos para ajustar una red son los siguientes:

- Agregue nuestra red personalizada además de una red base ya capacitada.

- Congele la red base.

- Entrena la parte que agregamos.

- Descongela algunas capas en la red base. (Tenga en cuenta que no debe descongelar las capas de “normalización por lotes”, que no son relevantes aquí ya que no existen tales capas en VGG16. La normalización por lotes y su impacto en el ajuste fino se explica en el siguiente capítulo).

- Entrenar conjuntamente ambas capas y la parte que agregamos.

![](./img/finetuning.png)


Ajustaremos las últimas tres capas convolucionales, lo que significa que todas las capas hasta `block4_pool` deben congelarse, y las capas `block5_conv1`, `block5_conv2` y `block5_conv3` deben poder entrenarse.

¿Por qué no ajustar más capas? ¿Por qué no ajustar toda la base convolucional? Tú podrías. Pero debes considerar lo siguiente:

- Las capas anteriores en la base convolucional codifican características más genéricas y reutilizables, mientras que las capas superiores codifican características más especializadas. Es más útil ajustar las funciones más especializadas, porque son las que deben reutilizarse en su nuevo problema. Se producirían rendimientos rápidamente decrecientes si se ajustaran las capas inferiores.

- Cuantos más parámetros entrenes, mayor será el riesgo de sobreajuste. La base convolucional tiene 15 millones de parámetros, por lo que sería arriesgado intentar entrenarla en su pequeño conjunto de datos.


![](./img/finetuning_2.png)

In [ ]:
# Imports que vamos a necesitar
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import VGG16, imagenet_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np

#Cargamos el dataset CIFAR10
(trainX, trainY), (testX, testY) = cifar10.load_data() 

# Normalizamos las entradas de idéntica forma a como lo hicieron para entrenar la VGG16 en imageNet
trainX = imagenet_utils.preprocess_input(trainX) 
testX = imagenet_utils.preprocess_input(testX) 

# Definimos dimensiones de nuestros datos de entrada y lista con las categorias de las clases
input_shape = (32, 32, 3) 
labelNames = ["Avión", "Automóvil", "Pájaro", "Gato", "Ciervo", "Perro", "Rana", "Caballo", "Barco", "Camión"] 

# En caso de inestabilidades numéricas pasar datos a one-hot encoding
trainY = to_categorical(trainY) 
testY = to_categorical(testY) 

# Importamos VGG16 con pesos de imagenet y sin top_model especificando tamaño de entrada de datos
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
# Mostramos la arquitectura
base_model.summary()

In [ ]:
# Congelamos las capas de los 4 primeros bloques convolucionales, el quinto se re-entrena
# En base_model.layers.name tenemos la información del nombre de la capa
from tensorflow.keras.optimizers import RMSprop

for layer in base_model.layers: 
    if layer.name == 'block3_conv1': 
        break 
    layer.trainable = False
    print('Capa ' + layer.name + ' congelada...') 
      

# Tomamos la última capa del model y le añadimos nuestro clasificador (top_model)
last = base_model.layers[-1].output 
x = Flatten()(last) 
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x) 
x = Dense(256, activation='relu', name='fc2')(x) 
x = Dense(10, activation='softmax', name='predictions')(x) 
model = Model(base_model.input, x) 


# Compilamos el modelo
model.compile(Adam(lr=0.0005), loss = "categorical_crossentropy", metrics=["accuracy"])

# Vamos a visualizar el modelo prestando especial atención en el número de pesos total y el número de pesos entrenables.
# ¿tiene sentido en comparación al ejemplo de transfer learning?
model.summary() 

In [ ]:
# Entrenamos el modelo
H = model.fit(trainX, 
              trainY, 
              validation_split=0.2, 
              batch_size=64, 
              epochs=20, 
              verbose=1) 

# Evaluación del modelo
print("[INFO]: Evaluando el modelo...")
predictions = model.predict(testX, batch_size=64) 
# Obtener el report de clasificación
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)) 

# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 20), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 20), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 20), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 20), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

# Summary

Las `convnets` son el mejor tipo de modelos de aprendizaje automático para tareas de visión por computadora. Es posible entrenar uno desde cero incluso con un conjunto de datos muy pequeño, con resultados decentes.

Los `convnets` funcionan aprendiendo una jerarquía de patrones y conceptos modulares para representar el mundo visual.

En un conjunto de datos pequeño, el problema principal será el `sobreajuste`. El aumento de datos es una forma poderosa de combatir el sobreajuste cuando se trabaja con datos de imágenes.

Es fácil reutilizar una `convnet` existente en un nuevo conjunto de datos mediante la extracción de funciones. Esta es una técnica valiosa para trabajar con conjuntos de datos de imágenes pequeños.

Como complemento a la extracción de características, se puede utilizar el `fine tuning`, que adapta a un nuevo problema algunas de las representaciones aprendidas previamente por un modelo existente. Esto lleva el rendimiento un poco más allá.

## References

- Deep Learning with Python, Second Edition